# Monads

When writing an app we define data structures that model our domain entities, e.g.

In [1]:
    record Wallet(int amount);

We then write our domain logic as `pure` functions, they are easier to write and test, and produce the same answer every time they are called, e.g. 

In [2]:
Wallet addTen(Wallet w1) => new Wallet(w1.amount + 10);
Console.WriteLine(addTen(new Wallet(10)));

Wallet { amount = 20 }


However in the real world we dont get to deal only with `pure` functions, our code typically includes `effects`.
Here are some examples of effects

```c#
// A user inputs a entity in the UI, its optional so they might not input anything
Wallet? getUserInput() => ...

// We get all the users wallets
IEnumerable<Wallet> getAllUsersWallets() => ...

// We get a wallet asynchronously
Task<Wallet> getWalletAsync() => ...

// We get a wallet from a computation that might error
public interface Result<T> { }
public record Success<T>(T value) : Result<T> { }
public record Error<T>(string message) : Result<T> { }
Result<Wallet> getWalletOrError() => ...

// there are more, state, environment, ...
```

Our domain types are still the important bit, our domain logic should be able to run on these types, but the extra effect structure makes it hard.
We dont want to litter our code dealing with these effects, its boilerplate, not unique to this application, only our domain logic is unique.

We can see that all these effect types `wrap` our domain types

```c#
Wallet? // sometimes Option<Wallet> or Maybe<Wallet>
IEnumerable<Wallet>
Task<Wallet>
Result<Wallet>
```

We just need to define how to `unwrap` the types to get at our core domain type. Then we can focus only on our domain logic.
There are several interfaces to implement `functor`, `applicative`, `monad`, `arrow` etc depending on what behaviour you want.

`Functor` is the simplest interface, it defines `map`, which takes a function `TWrapped -> TWrapped` and applys it to the wrapped value.


In [3]:
// if the wallet is null, return null, otherwise apply the function
T? mapNullable<T>(T? w, Func<T, T> f) => w == null ? default(T) : f(w);
Console.WriteLine((mapNullable(new Wallet(10), addTen), mapNullable<Wallet>(null, addTen)));


(Wallet { amount = 20 }, )


In [4]:

// apply the function to every item in the IEnumerable
IEnumerable<T> mapEnumerable<T>(IEnumerable<T> ts, Func<T, T> f) => ts.Select(f);
Console.WriteLine(string.Join(", ",mapEnumerable( new List<Wallet> { new Wallet(10), new Wallet(20) }, addTen)));


Wallet { amount = 20 }, Wallet { amount = 30 }


In [5]:
// apply the function to the value if it is a Success, otherwise return the Error
public interface Result<T> { }
public record Success<T>(T value) : Result<T> { }
public record Error<T>(string message) : Result<T> { }
Result<T> mapResult<T>(Result<T> w, Func<T, T> f) => w switch
{
    Success<T> s => new Success<T>(f(s.value)),
    Error<T> e => e
};
Console.WriteLine( mapResult(new Error<Wallet>("error"), addTen));
Console.WriteLine(mapResult(new Success<Wallet>(new Wallet(10)), addTen));

Error { message = error }
Success { value = Wallet { amount = 20 } }


Once you've defined `map` you can apply your pure domain logic to the results of `effects` (wrapped values) when ever you want. 

programs are created by composing functions together to make larger ones. passing the outputs of one function to the inputs of another

In [6]:
Wallet addFifty(Wallet w) => addTen(addTen(addTen(addTen(addTen(w)))));
Console.WriteLine(addFifty(new Wallet(10)));

Wallet { amount = 60 }


composing functions is syntatically equivalent to line by line c# style code e.g.

In [7]:
Wallet addFifty(Wallet w) {
    Wallet w1 = addTen(w);
    Wallet w2 = addTen(w1);
    Wallet w3 = addTen(w2);
    Wallet w4 = addTen(w3);
    Wallet w5 = addTen(w4);
    return w5;
}


Pure functions are easy to compose. To compose effects, you implement the  `monad` interface. The `bind` function is one way of doing it, given a `wrapped` type and an effect, how do you apply the effect to the `wrapped` value.

In [11]:
// If the result is a Success, apply the function to the value, otherwise return the Error
public static Result<T2> bind<T1, T2>(Result<T1> result, Func<T1, Result<T2>> f) => result switch
{
    Success<T1> s => f(s.value),
    Error<T1> e => new Error<T2>(e.message),
};
Console.WriteLine(bind(new Success<Wallet>(new Wallet(10)), a => new Success<Wallet>(addTen(a))));
Console.WriteLine(bind(new Success<Wallet>(new Wallet(10)), a => new Error<Wallet>("It broke")));


Success { value = Wallet { amount = 20 } }
Error { message = It broke }


Once you've declared `Monad/bind` for an effect, you can compose it freely

In [15]:
Result<Wallet> addTenIfSmallWallet(Wallet w) => w.amount > 75 ? new Error<Wallet>("It broke") : new Success<Wallet>(addTen(w));

Result<Wallet> addFiftyWithErrorHandling(Result<Wallet> w) {
    var w1 = bind<Wallet, Wallet>(w, addTenIfSmallWallet);
    var w2 = bind<Wallet, Wallet>(w1, addTenIfSmallWallet);
    var w3 = bind<Wallet, Wallet>(w2, addTenIfSmallWallet);
    var w4 = bind<Wallet, Wallet>(w3, addTenIfSmallWallet);
    var w5 = bind<Wallet, Wallet>(w4, addTenIfSmallWallet);
    return w5;
}
Console.WriteLine(addFiftyWithErrorHandling(new Success<Wallet>(new Wallet(10))));
Console.WriteLine(addFiftyWithErrorHandling(new Success<Wallet>(new Wallet(70))));

Success { value = Wallet { amount = 60 } }
Error { message = It broke }


on every line in this function we are checking if there is an error and short curcuiting if there is! compare this to Go where the idiom is to manually check

```go
func addFiftyWithErrorHandling(w Wallet) (Wallet, error) {

	w1, err := addTenIfSmallWallet(w)
	if err != nil {
		return fmt.Errorf(nil, err)
	}

	w2, err := addTenIfSmallWallet(w1)
	if err != nil {
		return fmt.Errorf(nil, err)
	}
    
    w3, err := addTenIfSmallWallet(w2)
	if err != nil {
		return fmt.Errorf(nil, err)
	}
    	
    w4, err := addTenIfSmallWallet(w3)
	if err != nil {
		return fmt.Errorf(nil, err)
	}
    	
    w5, err := addTenIfSmallWallet(w4)
	if err != nil {
		return fmt.Errorf(nil, err)
	}

    return w5, nil
}
```

Its much nicer in haskell, the language takes care of it, it would look like this

```haskell
addFiftyWithErrorHandling :: Monad m => a -> m a
addFiftyWithErrorHandling w = do
    w1 <- addTenIfSmallWallet w
    w2 <- addTenIfSmallWallet w1
    w3 <- addTenIfSmallWallet w2
    w4 <- addTenIfSmallWallet w3
    w5 <- addTenIfSmallWallet w4
    return w5
```

and this function can be called with any `monad` e.g. Result, Task, Maybe, List, etc